# Trabalho Prático II - POS Tagging

Autor: Pedro Henrique Madeira de Oliveira Pereira

> Observação: a função `train()` exige uma API key do `wandb`. No Google Colab, onde o trabalho foi feito, ela foi inserida como uma variável de ambiente secreta. Portanto, tenha isso em mente se tentar executar esse notebook.

## Importação de Dependências

Instalação das dependências do projeto. As principais dependências são: `numpy` (para computação numérica), `torch` (framework de deep learning), `datasets` (para carregar e manipular dados do Hugging Face), `transformers` (para carregar o modelo BERT, o tokenizador e o Trainer), `sklearn.metrics` (para calcular a acurácia e o relatório de classificação) e `google.colab.userdata` (para acessar chaves de API secretas no Colab).

In [4]:
!pip install datasets transformers[torch] scikit-learn numpy altair

import numpy as np
import torch
from datasets import load_dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)
from sklearn.metrics import accuracy_score, classification_report
import os
from google.colab import userdata
import pandas as pd
import altair as alt

## Verificação de GPU

Verifica se uma GPU (através da interface CUDA do PyTorch) está disponível. Se estiver, imprime uma mensagem de confirmação e define a variável `device` como "cuda"; caso contrário, define como "cpu". Isso garante que o treinamento utilize o hardware mais rápido disponível.

In [5]:
if torch.cuda.is_available():
    print(f"GPU detected.")
else:
    print("GPU was not detected. Training will occur on the CPU, which is slower.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

GPU detected.


## Carregamento do dataset

Aqui, é utilizada a biblioteca `datasets` para baixar e carregar o conjunto de dados "batterydata/pos_tagging" do Hugging Face Hub. Ao final, imprime a estrutura do dataset, mostrando os splits de treino (`train`) e teste (`test`), suas colunas e o número de linhas em cada um.

In [6]:
try:
    dataset = load_dataset("batterydata/pos_tagging")
    print("Dataset loaded:")
    print(dataset)
except Exception as e:
    print(f"Error while loading the dataset: {e}")
    exit()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/587 [00:00<?, ?B/s]

train.json: 0.00B [00:00, ?B/s]

test.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/13054 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1451 [00:00<?, ? examples/s]

Dataset loaded:
DatasetDict({
    train: Dataset({
        features: ['words', 'labels'],
        num_rows: 13054
    })
    test: Dataset({
        features: ['words', 'labels'],
        num_rows: 1451
    })
})


## Geração da lista de classes (labels)

Itera sobre todo o conjunto de dados (tanto treino quanto teste) para encontrar todas as etiquetas/classes (tags ou labels) únicas de Part-of-Speech (POS). Em seguida, cria uma lista ordenada (`label_list`) e dois dicionários de mapeamento, `id2label` (de ID para string) e `label2id` (de string para ID), que servem para converter as etiquetas em um formato que o modelo entende.

In [7]:
unique_labels = set()

for split in dataset:
    for label_list_i in dataset[split]['labels']:
        unique_labels.update(label_list_i)

label_list = sorted(list(unique_labels))

num_labels = len(label_list)
id2label = {i: label for i, label in enumerate(label_list)}
label2id = {label: i for i, label in enumerate(label_list)}

print(f"\nNum of labels: {num_labels}")
print("Labels list:")
print(label_list)


Num of labels: 48
Labels list:
['#', '$', "''", '(', ')', ',', '-LRB-', '-NONE-', '-RRB-', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']


## Inicialização do Tokenizador

Define o nome do modelo pré-treinado que servirá de base, "bert-base-cased". Em seguida, baixa e inicializa o `AutoTokenizer` correspondente a esse modelo. O tokenizador é responsável por dividir o texto em subtokens que o BERT entende.

In [8]:
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

## Definição da função de Pré-processamento

Define a função `tokenize_and_align_labels`, que é a etapa de pré-processamento mais importante. Esta função tokeniza as palavras de entrada e alinha as etiquetas (`labels`) aos novos subtokens. Como o BERT pode quebrar uma palavra em várias partes (ex: "running" -> "run", "##ning"), esta função garante que apenas o primeiro subtoken de cada palavra original receba a etiqueta de POS correta. Os demais subtokens e tokens especiais (como `[CLS]` e `[SEP]`) recebem a etiqueta `-100`, que é ignorada pela função de custo durante o treinamento.

In [9]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["words"],
                                 truncation=True,
                                 is_split_into_words=True,
                                 padding=False)

    labels = []
    for i, label_strings in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx == previous_word_idx:
                label_ids.append(-100)
            else:
                label_str = label_strings[word_idx]
                label_ids.append(label2id[label_str])

            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

## Aplicação da função de Pré-processamento

Aplica a função `tokenize_and_align_labels`  a todo o conjunto de dados (treino e teste) usando o método `.map()`. O argumento `batched=True` permite processar os dados em lotes, tornando essa etapa muito mais rápida. O resultado é um novo dataset, `tokenized_datasets`, pronto para o treinamento.

In [10]:
print("\Starting tokeinzation and alignment of labels...")
try:
    tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)
    print("Tokenization finished.")
except Exception as e:
    print(f"Error during tokenization: {e}")
    exit()

\Starting tokeinzation and alignment of labels...


<>:1: SyntaxWarning: invalid escape sequence '\S'
<>:1: SyntaxWarning: invalid escape sequence '\S'
/tmp/ipython-input-1895557343.py:1: SyntaxWarning: invalid escape sequence '\S'
  print("\Starting tokeinzation and alignment of labels...")


Map:   0%|          | 0/13054 [00:00<?, ? examples/s]

Map:   0%|          | 0/1451 [00:00<?, ? examples/s]

Tokenization finished.


## Inicialização do Modelo

Baixa e inicializa o modelo `AutoModelForTokenClassification` a partir do checkpoint "bert-base-cased". Crucialmente, ela passa os mapeamentos `id2label`, `label2id` e o `num_labels` (número total de classes). Isso informa à biblioteca que ela deve carregar o BERT, mas adicionar uma nova "cabeça" de classificação no topo, específica para a tarefa de classificação de tokens, com o número correto de saídas e pronta para ser treinada.

In [11]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Inicialização do data_collator

Inicializa um `DataCollatorForTokenClassification`. Esse objeto é um utilitário que o `Trainer` usará para agrupar os exemplos do dataset em lotes (batches). Sua principal função é aplicar padding dinamicamente a cada lote, garantindo que todas as sentenças e suas respectivas labels em um mesmo lote tenham o mesmo comprimento, o que é necessário para o treinamento em GPU.

In [12]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

## Definição da função para computação de métricas

Define a função `compute_metrics`, que será chamada pelo `Trainer` ao final de cada época de avaliação. Ela recebe as previsões brutas do modelo (logits) e as labels verdadeiras, converte os logits para a classe prevista (usando `np.argmax`), filtra todas as etiquetas de alinhamento (`-100`) e, por fim, calcula e retorna a acurácia geral das previsões.

In [13]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [p for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [l for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    true_predictions = [item for sublist in true_predictions for item in sublist]
    true_labels = [item for sublist in true_labels for item in sublist]

    return {
        "accuracy": accuracy_score(true_labels, true_predictions)
    }

## Obtenção da API Key

Acessa o gerenciador de "secrets" do Google Colab para buscar uma chave de API armazenada como `WANDB_KEY`. Em seguida, define essa chave como uma variável de ambiente. Isso permite que o `Trainer` se autentique automaticamente com o serviço Weights & Biases (wandb) para registrar métricas de treinamento.

In [14]:
wandb_key = userdata.get('WANDB_KEY')

os.environ["WANDB_API_KEY"] = wandb_key

## Definição dos hiperparâmetros e configurações de treinamento

Define todos os hiperparâmetros de treinamento usando a classe `TrainingArguments`. Isso inclui o diretório onde os modelos serão salvos, a frequência de avaliação, a taxa de aprendizado, o tamanho dos lotes, o número de épocas e a instrução `load_best_model_at_end=True`, que garante que o melhor modelo (e não apenas o último) seja mantido ao final do processo.

In [15]:
training_args = TrainingArguments(
    output_dir="./results_pos_tagging",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    save_strategy="epoch",
    load_best_model_at_end=True,
    seed=42
)

## Instanciação do Trainer

Inicializa o objeto `Trainer`. Esta é a classe principal do Hugging Face que orquestra todo o processo de fine-tuning. Ela recebe o modelo, os argumentos de treino, os datasets de treino e teste já tokenizados, o tokenizador, o data collator e a função de métricas.

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-3760727303.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


## Treinamento do modelo

Inicia o processo de treinamento (fine-tuning) chamando o método `.train()`. Isso executará o treinamento em um número determinado de épocas, validando o modelo no conjunto de teste ao final de cada uma, registrando os resultados no Weights & Biases e salvando o melhor modelo.

In [17]:
print("Training started")
trainer.train()
print("Training finished")

Training started


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Currently logged in as: madeirapedrohenrique1112 (madeirapedrohenrique1112-ufmg-nlp) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,0.085500,0.101628,0.972554
2,0.055900,0.088738,0.976347
3,0.041200,0.088680,0.977163


Training finished


## Avaliação do modelo

Chama explicitamente o método `.evaluate()` no conjunto de teste. Como `load_best_model_at_end=True` foi definido nos argumentos, o `Trainer` automaticamente usa o melhor checkpoint salvo durante o treinamento para a avaliação final.

In [18]:
print("Evaluating general accuracy")
eval_results = trainer.evaluate(tokenized_datasets["test"])

print(f"General accuracy on the test set: {eval_results['eval_accuracy']:.4f}")

Evaluating general accuracy


General accuracy on the test set: 0.9772


## Previsões no conjunto de Teste

Usa o `Trainer` treinado para fazer previsões (`.predict()`) em todo o conjunto de teste. Isso retorna os logits (saídas brutas do modelo), que são então convertidos em índices de classe (o ID da etiqueta prevista) aplicando `np.argmax` no eixo 2.

In [19]:
print("Generating provisions for the test set")
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

Generating provisions for the test set


## Dicionário de Labels

Define um dicionário que mapeia os códigos curtos das etiquetas de POS (ex: 'NN', 'VBZ') para seus nomes semânticos completos e legíveis. Em seguida, cria uma lista com esses nomes, na ordem correta, para ser usada no relatório de classificação.

In [20]:
penn_treebank_map = {
    "#": "Pound Sign / Hash Symbol",
    "$": "Dollar Sign Symbol",
    "''": "Closing Quotation Mark",
    "(": "Opening Parenthesis",
    ")": "Closing Parenthesis",
    ",": "Comma",
    "-LRB-": "Left Round Bracket (ASCII)",
    "-NONE-": "Null Element (syntactic trace)",
    "-RRB-": "Right Round Bracket (ASCII)",
    ".": "Sentence Terminator (Period)",
    ":": "Colon",
    "``": "Opening Quotation Mark",
    "CC": "Coordinating Conjunction", # (e.g., 'and', 'but')
    "CD": "Cardinal Number", # (e.g., '1', 'two')
    "DT": "Determiner", # (e.g., 'the', 'a', 'this')
    "EX": "Existential 'there'", # (e.g., 'there is')
    "FW": "Foreign Word",
    "IN": "Preposition or Subordinating Conjunction",
    "JJ": "Adjective", # (e.g., 'big', 'cold')
    "JJR": "Adjective, Comparative", # (e.g., 'bigger', 'colder')
    "JJS": "Adjective, Superlative", # (e.g., 'biggest', 'coldest')
    "LS": "List Item Marker", # (e.g., '1)', 'A.')
    "MD": "Modal Verb", # (e.g., 'can', 'will', 'should')
    "NN": "Noun, Singular or Mass", # (e.g., 'cat', 'tree')
    "NNP": "Proper Noun, Singular", # (e.g., 'Google')
    "NNPS": "Proper Noun, Plural", # (e.g., 'Vikings')
    "NNS": "Noun, Plural", # (e.g., 'cats', 'trees')
    "PDT": "Predeterminer", # (e.g., 'all', 'both' in 'all the cats')
    "POS": "Possessive Ending", # (e.g., \"'s\")
    "PRP": "Personal Pronoun", # (e.g., 'I', 'he', 'she')
    "PRP$": "Possessive Pronoun", # (e.g., 'my', 'your', 'his')
    "RB": "Adverb", # (e.g., 'quickly', 'very')
    "RBR": "Adverb, Comparative", # (e.g., 'faster')
    "RBS": "Adverb, Superlative", # (e.g., 'fastest')
    "RP": "Particle", # (e.g., 'up' in 'look up', 'out' in 'carry out')
    "SYM": "Symbol", # (e.g., '+', '%', '&')
    "TO": "Preposition 'to' (or infinitive marker)",
    "UH": "Interjection", # (e.g., 'Wow!', 'Oh', 'Hey')
    "VB": "Verb, Base Form", # (e.g., 'eat', 'run')
    "VBD": "Verb, Past Tense", # (e.g., 'ate', 'ran')
    "VBG": "Verb, Gerund or Present Participle", # (e.g., 'eating', 'running')
    "VBN": "Verb, Past Participle", # (e.g., 'eaten', 'run')
    "VBP": "Verb, Present Tense (non-3rd person singular)", # (e.g., 'I eat', 'we run')
    "VBZ": "Verb, Present Tense (3rd person singular)", # (e.g., 'he eats', 'she runs')
    "WDT": "Wh-Determiner", # (e.g., 'which', 'that')
    "WP": "Wh-Pronoun", # (e.g., 'who', 'what')
    "WP$": "Possessive Wh-Pronoun", # (e.g., 'whose')
    "WRB": "Wh-Adverb", # (e.g., 'where', 'when')
}

semantic_target_names = [penn_treebank_map.get(tag, tag) for tag in label_list]

## Pós-processamento das previsões e labels

Realiza o pós-processamento final das previsões e labels. Essa célula itera sobre os resultados, remove as etiquetas de alinhamento, converte os IDs numéricos de volta para suas etiquetas de string e "achata" as listas de listas em duas listas únicas: `flat_true_predictions` e `flat_true_labels`.

In [21]:
true_predictions_str = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels_str = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
flat_true_predictions = [item for sublist in true_predictions_str for item in sublist]
flat_true_labels = [item for sublist in true_labels_str for item in sublist]

## Relatório de classificação

Usa a função `classification_report` do scikit-learn para gerar e imprimir um relatório detalhado. O relatório compara as previsões e as etiquetas verdadeiras (ambas na lista achatada), mostrando métricas de precisão, recall e f1-score para cada classe de POS individualmente, usando os nomes semânticos legíveis definidos anteriormente.

In [22]:
print("Detailed Classification Report by Label (Test)")

report = classification_report(
    flat_true_labels,
    flat_true_predictions,
    labels=label_list,
    target_names=semantic_target_names,
    digits=4,
    zero_division=0 # to avoid warnings
)
print(report)

Detailed Classification Report by Label (Test)
                                               precision    recall  f1-score   support

                     Pound Sign / Hash Symbol     1.0000    1.0000    1.0000        15
                           Dollar Sign Symbol     1.0000    1.0000    1.0000       329
                       Closing Quotation Mark     0.9952    1.0000    0.9976       208
                          Opening Parenthesis     0.0000    0.0000    0.0000         0
                          Closing Parenthesis     0.0000    0.0000    0.0000         0
                                        Comma     1.0000    1.0000    1.0000      1790
                   Left Round Bracket (ASCII)     0.9792    0.8868    0.9307        53
               Null Element (syntactic trace)     1.0000    1.0000    1.0000      2486
                  Right Round Bracket (ASCII)     0.9636    0.9464    0.9550        56
                 Sentence Terminator (Period)     1.0000    1.0000    1.0000      

## Geração de DataFrame para plotagem de gráficos

Obtém o classification report no formato de dicionário e gera os dataframes que serão usados para a plotagem dos gráficos.

In [23]:
report_dict = classification_report(
    flat_true_labels,
    flat_true_predictions,
    labels=label_list,
    target_names=semantic_target_names,
    digits=4,
    zero_division=0,
    output_dict=True
)

df_report = pd.DataFrame(report_dict).transpose()

df_report = df_report.reset_index().rename(columns={'index': 'label'})

summary_rows = ['accuracy', 'macro avg', 'weighted avg']
df_report_cleaned = df_report[~df_report['label'].isin(summary_rows)].copy()

df_report_cleaned['support'] = df_report_cleaned['support'].astype(int)

df_with_support = df_report_cleaned[df_report_cleaned['support'] > 0].copy()

## Gráfico 1 - Precisão para cada label

Mostra o ranking das labels que tiveram maior precisão na sua avaliação. É possível observar que para muitas delas a precisão ficou muito próxima de 1 (100%), e para a grande maioria ficou acima de 80%. Houveram 3 labels que obtiveram precisão 0, causado principalmente pela falta de amostra, já que a frequência delas foi muito baixa (como poderá ser observado no próximo gráfico).

In [30]:
chart_ranking = alt.Chart(df_with_support).mark_bar().encode(
    x=alt.X('precision:Q', title='Precision', scale=alt.Scale(domain=[0, 1.05])),
    y=alt.Y('label:N', title='Label', sort='-x'),
    tooltip=['label', 'precision', 'support']
).properties(
    title=f'Labels ranked by precision (support > 0)'
).interactive()

chart_ranking

alt.Chart(...)

## Gráfico 2 - Frequência de cada label no set de teste

Mostra a frequência de palavras pertencentes a cada label no set de testes. A frequência foi de mais de 5000 ocorrências para substantivos, e muito próximo ou igual a 1 para classes como interjeição e palavra estrangeira.

In [31]:
chart_support = alt.Chart(df_with_support).mark_bar().encode(
    x=alt.X('support:Q', title='Frequency (Support) in Test Set'),

    y=alt.Y('label:N', title='Part-of-Speech Tag', sort='-x'),

    tooltip=['label', 'support', 'f1-score']
).properties(
    title='Frequency of each label in Test Set (support > 0)'
).interactive()

chart_support

alt.Chart(...)

## Gráfico 3 - Comparação entre precisão e frequência

Mostra uma comparação mais direta entre a precisão e a frequência de determinada classe de palavras. A classe com maior frequência, que é a de substantivos, ficou no meio do ranking de precisão, mas ainda mantendo uma precisão elevada.

In [35]:
chart_combined = alt.Chart(df_with_support).mark_bar().encode(
    y=alt.Y('label:N',
          title='Label',
          sort='-x'),

    x=alt.X('precision:Q',
          title='Precision',
          scale=alt.Scale(domain=[0, 1.05])),

    color=alt.Color('support:Q',
                  title='Support (Frequency)',
                  scale=alt.Scale(range='heatmap')),

    tooltip=['label', 'precision', 'support', 'f1-score']
).properties(
    title='Label precision (bar length) vs. frequency (bar color)'
).interactive()

chart_combined

alt.Chart(...)

## Conclusão

O modelo atingiu um desempenho geral excelente, acertando aproximadamente 97% de todas as palavras no conjunto de teste (accuracy). O modelo é extremamente confiável para identificar as classes gramaticais mais frequentes, como substantivos, pontuação e preposições. Por causa disso, a média ponderada também ficou alta, bem próxima à acurácia geral.

O principal ponto de atenção é a média macro, que cai para aproximadamente 82%. Esta média é baixa porque ela trata todas as classes com o mesmo peso, e o modelo teve desempenho zero em várias classes extremamente raras. Além dessas classes raras, as maiores dificuldades reais do modelo foram identificar as classes "Proper Noun, Plural" (nomes próprios no plural) e "Adverb, Superlative" (advérvios superlativos, como "fastest"). Este último teve uma precisão muito baixa (aprox. 48%), mas um recall alto (aprox. 84%). Isso significa que, embora o modelo tenha conseguido encontrar a maioria dos advérbios superlativos reais que existiam no teste, ele também classificou incorretamente muitas outras palavras (falsos positivos) como se fossem superlativos.